<a href="https://colab.research.google.com/github/VadPA/PyTorch/blob/master/Lesson_3_hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [267]:
import numpy as np
import torch
from torch.utils.data import Dataset
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from torch import nn
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler

In [268]:
class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 2 * hidden_dim)
        self.fc2 = nn.Linear(2 * hidden_dim, hidden_dim)
        self.bn = nn.BatchNorm1d(hidden_dim)
        self.bn2 = nn.BatchNorm1d(2 * hidden_dim)
        self.dp = nn.Dropout(0.25)
        self.fc3 = nn.Linear(1 * hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = F.leaky_relu(x)
        x = self.fc2(x)
        x = F.leaky_relu(x)
        x = self.fc3(x)
        x = F.leaky_relu(x)
        x = self.fc4(x)

        return x

In [269]:
class CustomDataset(Dataset):
    def __init__(self, is_train=True):
        super(CustomDataset, self).__init__()
        self.is_train = is_train
        self.sc = MinMaxScaler()
        self.X, self.Y = fetch_california_housing(return_X_y=True, as_frame=True)
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.Y, test_size=0.25,
                                                                                random_state=13)

    def __len__(self):
        if self.is_train:
            return len(self.X_train)
        else:
            return len(self.X_test)

    def __getitem__(self, idx):
        if self.is_train:
            x = np.array(self.X_train).astype(np.float32)
            y = np.array(self.y_train).astype(np.float32)
            x = self.sc.fit_transform(x)
            return x[idx], y[idx]
        else:
            x = np.array(self.X_test).astype(np.float32)
            y = np.array(self.y_test).astype(np.float32)
            x = self.sc.fit_transform(x)
            return x[idx], y[idx]

In [270]:
X_train = CustomDataset(is_train=True)

In [271]:
train_loader = torch.utils.data.DataLoader(dataset=X_train,
                                           batch_size=64,
                                           shuffle=True)

In [272]:
X_test = CustomDataset(is_train=False)

In [273]:
test_loader = torch.utils.data.DataLoader(dataset=X_test,
                                          batch_size=64,
                                          shuffle=True)

In [274]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [275]:
net = Net(input_dim=8, hidden_dim=32, output_dim=1).to(device)  # определить сеть
print(net)  # структура сети печати

Net(
  (fc1): Linear(in_features=8, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (bn): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dp): Dropout(p=0.25, inplace=False)
  (fc3): Linear(in_features=32, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=1, bias=True)
)


In [276]:
loss_func = torch.nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

In [277]:
num_epochs = 10

In [278]:
for epoch in range(num_epochs):
    net.train()
    training_loss = 0.0
    training_items = 0.0
    valid_loss = 0.0

    for i, data in enumerate(train_loader):
        inputs, labels = data[0], data[1]

        # Обнуляем градиент
        optimizer.zero_grad()
        # Делаем предсказание
        outputs = net(inputs)
        # Рассчитываем лосс-функцию
        loss = loss_func(outputs.mH[0, :], labels)
        # Делаем шаг назад по лоссу
        loss.backward()
        # Делаем шаг нашего оптимайзера
        optimizer.step()

        # выводим статистику о процессе обучения
        training_loss += loss.item()
        training_items += len(labels)
        # печатаем каждые 20 mini-batches
        if i % 20 == 0:
            net.eval()
            print(f'Epoch [{epoch + 1}/{num_epochs}]. '
                  f'Step [{i + 1}/{len(train_loader)}]. '
                  f'Train_Loss: {training_loss / training_items:.4f}', end='. ')
            training_loss, training_items = 0.0, 0.0

            valid_loss, loss_val = 0.0, 0.0
            for j, batch in enumerate(test_loader):
              inputs, targets = batch
              output = net(inputs)
              loss_val = loss_func(output.mH[0, :], targets)
              valid_loss += loss_val.item()
            valid_loss /= len(test_loader)

            print('Valid_Loss: {:.4f}'.format(valid_loss))
            net.train()



Epoch [1/10]. Step [1/242]. Train_Loss: 0.1073. Valid_Loss: 5.0022
Epoch [1/10]. Step [21/242]. Train_Loss: 0.0720. Valid_Loss: 4.1715
Epoch [1/10]. Step [41/242]. Train_Loss: 0.0570. Valid_Loss: 2.6211
Epoch [1/10]. Step [61/242]. Train_Loss: 0.0290. Valid_Loss: 1.3529
Epoch [1/10]. Step [81/242]. Train_Loss: 0.0187. Valid_Loss: 1.2510
Epoch [1/10]. Step [101/242]. Train_Loss: 0.0188. Valid_Loss: 1.1814
Epoch [1/10]. Step [121/242]. Train_Loss: 0.0173. Valid_Loss: 1.1122
Epoch [1/10]. Step [141/242]. Train_Loss: 0.0159. Valid_Loss: 1.0180
Epoch [1/10]. Step [161/242]. Train_Loss: 0.0150. Valid_Loss: 0.9165
Epoch [1/10]. Step [181/242]. Train_Loss: 0.0128. Valid_Loss: 0.8036
Epoch [1/10]. Step [201/242]. Train_Loss: 0.0103. Valid_Loss: 0.6807
Epoch [1/10]. Step [221/242]. Train_Loss: 0.0097. Valid_Loss: 0.6195
Epoch [1/10]. Step [241/242]. Train_Loss: 0.0096. Valid_Loss: 0.5936
Epoch [2/10]. Step [1/242]. Train_Loss: 0.0072. Valid_Loss: 0.5897
Epoch [2/10]. Step [21/242]. Train_Loss: 0

In [312]:
data_iter = iter(test_loader)
inputs, labels = data_iter.next()

In [313]:
net.eval()
outputs = net(inputs)

In [314]:
print(outputs.T)

tensor([[5.0002, 0.9242, 3.0311, 2.9310, 1.0628, 1.1833, 2.2842, 0.8712, 4.5034,
         1.3742, 1.3403, 2.1605, 0.9459, 1.3707, 1.8020, 1.0927, 1.5268, 4.4358,
         2.1857, 1.5327, 3.1582, 0.8906, 1.6457, 1.4832, 1.6531, 1.1229, 1.3531,
         3.2579, 1.2487, 2.8839, 1.1571, 1.1721, 1.4059, 3.0556, 4.2594, 1.7353,
         1.3838, 2.3092, 0.7808, 2.5013, 2.9113, 0.6336, 1.6473, 0.9617, 1.0746,
         3.3883, 1.0890, 1.3062, 1.5301, 2.1855, 0.9084, 0.8614, 1.0218, 2.4872,
         0.9049, 1.9236, 0.9395, 1.0544, 1.3032, 1.0063, 1.4342, 0.9803, 1.7111,
         1.7455]], grad_fn=<PermuteBackward0>)


In [315]:
print(loss_func(outputs.mH[0, :], labels))

tensor(0.4374, grad_fn=<MseLossBackward0>)


In [316]:
print(labels)

tensor([5.0000, 0.6780, 4.4110, 3.4120, 1.1320, 0.8650, 2.2710, 0.8110, 5.0000,
        1.2830, 1.6440, 4.5000, 1.1180, 1.0940, 1.7130, 1.1740, 1.2980, 5.0000,
        3.1430, 3.7610, 4.1390, 0.7800, 2.6350, 1.0620, 1.8140, 1.8750, 1.5670,
        3.3110, 1.2910, 1.8750, 0.7410, 1.5160, 1.9750, 2.6470, 5.0000, 1.8750,
        2.1190, 2.4460, 0.4930, 3.2600, 3.5970, 0.5910, 3.2730, 0.8750, 0.9680,
        3.8600, 1.1230, 1.9430, 0.8450, 2.5390, 1.1790, 0.5880, 0.5800, 3.0080,
        0.6930, 1.3820, 1.5740, 1.3590, 1.0110, 0.8940, 1.7770, 1.0850, 0.9720,
        2.4940])


In [284]:
# !pip install torchmetrics

In [285]:
from torchmetrics import R2Score

In [286]:
r2score = R2Score()

In [317]:
r2score(outputs[:,0], labels)

tensor(0.7370, grad_fn=<SqueezeBackward0>)